In [1]:
import warnings 
warnings.filterwarnings("ignore")

In [2]:
%run ./analysis_utils/log_retriever.py

In [31]:
baseline_log_retriever = LogRetreiver(model="baseline", test_path="../../data/test/merged_test_data.csv")
_, baseline_pred_dfs = baseline_log_retriever.get_logs()

chunk_log_retriever = LogRetreiver(model="chunk_combinations", test_path="../../data/test/merged_test_data.csv")
_, chunk_pred_dfs = chunk_log_retriever.get_logs()

text_summarizer_log_retriever = LogRetreiver(model="text_summarizer", test_path="../../data/test/merged_test_data.csv")
_, text_summarizer_pred_dfs = text_summarizer_log_retriever.get_logs()

In [32]:
def merge_with_test(pred_dfs):
    merged_dfs = []
    test_df = pd.read_csv("../../data/test/merged_test_data.csv")
    for pred_df in pred_dfs:
        merged_df = pd.merge(test_df, pred_df, on="pair_id")
        if "overall_x" in merged_df.columns:
            merged_df = merged_df.rename(columns={"overall_x": "overall"})
            merged_df = merged_df.drop(columns=["overall_y"])
        merged_dfs.append(merged_df)

    return merged_dfs

def get_avg_dist(merged_dfs, model="baseline"):
    avg_dists = []
    for merged_df in merged_dfs:
        if model != "chunk_combinations":
            merged_df["dist"] = abs(merged_df["overall"] - merged_df["pred"])
        else:
            merged_df["dist"] = abs(merged_df["overall"] - merged_df["logits"])
        avg_dist = merged_df["dist"].mean()
        avg_dists.append(avg_dist)

    avg_dist = sum(avg_dists) / len(avg_dists)
    return avg_dist

def run(pred_dfs, model="baseline"):
    merged_dfs = merge_with_test(pred_dfs)
    avg_dist = get_avg_dist(merged_dfs, model=model)

    return avg_dist

In [33]:
baseline_avg_dist = run(baseline_pred_dfs, model="baseline")
chunk_avg_dist = run(chunk_pred_dfs, model="chunk_combinations")
text_summarizer_avg_dist = run(text_summarizer_pred_dfs, model="text_summarizer")

print(f"Baseline avg dist: {baseline_avg_dist}")
print(f"Chunk avg dist: {chunk_avg_dist}")
print(f"Text summarizer avg dist: {text_summarizer_avg_dist}")

Baseline avg dist: 0.7012011087138715
Chunk avg dist: 0.7716308341652507
Text summarizer avg dist: 0.7389031005472665
